In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder\
    .appName('kafka-spark-kafka')\
    .master('local[2]')\
    .config('spark.sql.shuffle.partitions',2)\
    .getOrCreate()

### Connecting to ZooKeeper Using
* zookeeper-server-start.sh $KAFKA_HOME/config/zookeeper.properties 

### Creating A Kafka Topic Named SEEDHE_MAUT On Kafka using
* kafka-topics.sh --zookeeper localhost 2181 --create --topic seedhe_maut --replication-factor 1 --partitions 2


### Activating the Kafka Server
* kafka-server-start.sh $KAFKA_HOME/config/server.properties



### Continously downloaded unbounded data using tweepy would be streamed to a kafka topic named 'seedhe_maut'

### Genrating the tweets using the TWEEPY API for Twitter and Devloper Access and Then sending those tweets to kafka  and can be seen using :
* kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic seedhe_maut


In [3]:
df = spark.readStream\
    .format('kafka')\
    .option('kafka.bootstrap.servers', 'localhost:9092')\
    .option('subscribe','seedhe_maut')\
    .load()

In [4]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
# processing = df.selectExpr("CAST(value as STRING) AS value")\
#     .select(from_json('value','id STRING, time STRING, text STRING').alias('x'))\
#     .selectExpr("CAST(from_unixtime(x.time,'HH:mm:ss') as TIMESTAMP) as time","explode(split(lower(x.text),' ')) as hashtag")\
#     .where("length(hashtag) > 1")\
#     .where("substring(hashtag,1,1) = '#'")\
#     .groupBy("hashtag", window("time", windowDuration='60 minute')).agg(count('hashtag').alias('count'))\
#     .selectExpr("(hashtag, count, window.start as start, window.end as end) as result")\
#     .select(to_json('result').alias('value'))

In [6]:
processing = df.selectExpr("CAST(value as STRING) AS value")\
    .select(from_json('value','id STRING, time STRING, text STRING').alias('x'))\
    .selectExpr("CAST(from_unixtime(x.time,'HH:mm:ss') as TIMESTAMP) as time","explode(split(lower(x.text),' ')) as hashtag")\
    .where("length(hashtag) > 1")\
    .where("substring(hashtag,1,1) = '#'")\
    .selectExpr("(time, hashtag) as result")\
    .select(to_json('result').alias('value'))

In [7]:
processing.writeStream\
    .outputMode('append')\
    .format('console')\
    .option('truncate','false')\
    .start()

In [8]:
# query.awaitTermination()

### Creating A Kafka topice to produce this cleaned data of hashtags to
* kafka-topics.sh --zookeeper localhost 2181 --create --topic seedhe_maut_capturePoint --replication-factor 1 --partitions 2


### Consumer Cross Validate Debugging Command
* kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic seedhe_maut_capturePoint

In [9]:
processing.writeStream\
    .format('kafka')\
    .outputMode('append')\
    .option('kafka.bootstrap.servers','localhost:9092')\
    .option('topic','seedhe_maut_capturePoint')\
    .option('checkpointLocation','/home/sunbeam/Documents/tweepy/checkpoint2/')\
    .start()

In [ ]:
spark.streams.awaitAnyTermination()

In [1]:
spark.stop()

NameError: name 'spark' is not defined